# Лабораторная работа №5. Выполнил Акинин Александр, студент 431 группы

In [1]:
import numpy as np

matrix_dtype = np.float64
epsilon = 1e-15

n = 99
a_i, b_i, c_i, p_i = 1, 10, 1, 1
f = np.array([i+1 for i in range(n+1)], dtype=matrix_dtype)
round_digit = 4

### Функция для создания матрицы, данной в задании

In [2]:
def generate_matrix(n, a_i, b_i, c_i, p_i, f):
    m = n + 1
    matrix = np.zeros((m,m+1), dtype=matrix_dtype)
    
    for i in range(m):
        
        # Заполнить главную диагональ (b_i)
        matrix[i,i] = b_i
        
        # Заполнить элементы под главной диагональю (a_i)
        try: matrix[i,i-1] = a_i
        except: pass
        
        # Заполнить элементы над главной диагональю (c_i)
        try: matrix[i,i+1] = c_i
        except: pass
    
    # Заполнить последнюю строку (p_i)
    for i in range(m):
        matrix[-1, i] = p_i
    
    # Заполнить столбец свободных членов (f_i)
    for i in range(m):
        matrix[:, -1] = f
    
    return matrix

### Вспомогательные функции

In [3]:
def is_zero(element):
    return (abs(element) < epsilon)

def norm(matr):
    return np.sqrt(np.sum(matr**2))

### Создание матрицы СЛАУ

In [4]:
matrix = generate_matrix(n, a_i, b_i, c_i, p_i, f)

# СЛАУ: A_matr * x_vect = b_vect
A_matr = matrix[:n+1,:n+1]
b_vect = matrix[:,-1]

print('Изначальная матрица СЛАУ:\n', matrix)

Изначальная матрица СЛАУ:
 [[ 10.   1.   0. ...   0.   0.   1.]
 [  1.  10.   1. ...   0.   0.   2.]
 [  0.   1.  10. ...   0.   0.   3.]
 ...
 [  0.   0.   0. ...   1.   0.  98.]
 [  0.   0.   0. ...  10.   1.  99.]
 [  1.   1.   1. ...   1.   1. 100.]]


### 1. Метод Гаусса

In [5]:
# Прямой ход
def row_reduction_forward(matr):
    matrix = np.copy(matr)
    n = matrix.shape[0]
    
    for j in range(n-1):
        for i in range(j, n-1):
            if is_zero(matrix[j, j]):
                continue
            matrix[i+1, :] -= matrix[j, :] * matrix[i+1, j]/matrix[j, j]
    for i in range(n):
        matrix[i,:] /= matrix[i,i]
    return matrix


# Прямой и обратный ход
def rref(matr):
    matrix = np.copy(matr)
    n = matrix.shape[0]
    
    for j in range(n-1):
        for i in range(j, n-1):
            if is_zero(matrix[j, j]):
                continue
            matrix[i+1, :] -= matrix[j, :] * matrix[i+1, j]/matrix[j, j]
    last_row = np.copy(matrix[-1,:])
    for j in range(n-1, -1, -1):
        for i in range(j, -1, -1):
            if is_zero(matrix[j, j]):
                continue
            matrix[i-1, :] -= matrix[j, :] * matrix[i-1, j]/matrix[j, j]
    matrix[-1,:] = last_row
    for i in range(n):
        matrix[i,:] /= matrix[i,i]
    
    return matrix


# Получить последний столбец матрицы
def get_solution(reduced_matrix):
    return np.copy(reduced_matrix[:,-1])

In [6]:
solution_Gauss = get_solution(rref(matrix))
print('Решение СЛАУ методом Гаусса:\n', solution_Gauss)
print()

residual_Gauss = np.dot(A_matr, solution_Gauss)-b_vect
print('Вектор невязки для метода Гаусса:\n', residual_Gauss)
print()

residual_Gauss_norm = norm(residual_Gauss)
print('Норма вектора невязки для метода Гаусса:\n', residual_Gauss_norm)

Решение СЛАУ методом Гаусса:
 [ 8.33333333e-02  1.66666667e-01  2.50000000e-01  3.33333333e-01
  4.16666667e-01  5.00000000e-01  5.83333333e-01  6.66666667e-01
  7.50000000e-01  8.33333333e-01  9.16666667e-01  1.00000000e+00
  1.08333333e+00  1.16666667e+00  1.25000000e+00  1.33333333e+00
  1.41666667e+00  1.50000000e+00  1.58333333e+00  1.66666667e+00
  1.75000000e+00  1.83333333e+00  1.91666667e+00  2.00000000e+00
  2.08333333e+00  2.16666667e+00  2.25000000e+00  2.33333333e+00
  2.41666667e+00  2.50000000e+00  2.58333333e+00  2.66666667e+00
  2.75000000e+00  2.83333333e+00  2.91666667e+00  3.00000000e+00
  3.08333333e+00  3.16666667e+00  3.25000000e+00  3.33333333e+00
  3.41666667e+00  3.50000000e+00  3.58333333e+00  3.66666667e+00
  3.75000000e+00  3.83333333e+00  3.91666667e+00  4.00000000e+00
  4.08333333e+00  4.16666667e+00  4.25000000e+00  4.33333333e+00
  4.41666667e+00  4.50000000e+00  4.58333333e+00  4.66666667e+00
  4.75000000e+00  4.83333333e+00  4.91666667e+00  5.00000000

### 2. Метод Якоби

In [7]:
def Jacobi_method(A, b, norm_limit=1e-13):
    n = A.shape[0]
    
    D = np.diag(np.diag(A)) # только элементы главной диагонали A
    D_inv = np.diag(1/np.diag(A))
    
    B = np.dot(D_inv, (D-A))
    g = np.dot(D_inv, b)
    
    curr_x = np.ones(n)
    prev_x = np.copy(curr_x) + 1e10
    while norm(curr_x-prev_x) > norm_limit:
        prev_x = np.copy(curr_x)
        curr_x = np.dot(B, curr_x) + g

    return curr_x

In [8]:
solution_Jacobi = Jacobi_method(A_matr, b_vect)
print('Решение СЛАУ методом Якоби:\n', solution_Jacobi)
print()

residual_Jacobi = np.dot(A_matr, solution_Jacobi)-b_vect
print('Вектор невязки для метода Якоби:\n', residual_Jacobi)
print()

residual_Jacobi_norm = norm(residual_Jacobi)
print('Норма вектора невязки для метода Якоби:\n', residual_Jacobi_norm)

Решение СЛАУ методом Якоби:
 [ 8.33333333e-02  1.66666667e-01  2.50000000e-01  3.33333333e-01
  4.16666667e-01  5.00000000e-01  5.83333333e-01  6.66666667e-01
  7.50000000e-01  8.33333333e-01  9.16666667e-01  1.00000000e+00
  1.08333333e+00  1.16666667e+00  1.25000000e+00  1.33333333e+00
  1.41666667e+00  1.50000000e+00  1.58333333e+00  1.66666667e+00
  1.75000000e+00  1.83333333e+00  1.91666667e+00  2.00000000e+00
  2.08333333e+00  2.16666667e+00  2.25000000e+00  2.33333333e+00
  2.41666667e+00  2.50000000e+00  2.58333333e+00  2.66666667e+00
  2.75000000e+00  2.83333333e+00  2.91666667e+00  3.00000000e+00
  3.08333333e+00  3.16666667e+00  3.25000000e+00  3.33333333e+00
  3.41666667e+00  3.50000000e+00  3.58333333e+00  3.66666667e+00
  3.75000000e+00  3.83333333e+00  3.91666667e+00  4.00000000e+00
  4.08333333e+00  4.16666667e+00  4.25000000e+00  4.33333333e+00
  4.41666667e+00  4.50000000e+00  4.58333333e+00  4.66666667e+00
  4.75000000e+00  4.83333333e+00  4.91666667e+00  5.00000000e

### 3. Найти $\lambda_{\min}$ и $\lambda_{\max}$

In [9]:
eigenval, eigenvect = np.linalg.eig(A_matr)
print(f'𝜆min = {min(eigenval)}, 𝜆max = {max(eigenval)}')

𝜆min = 0.8998885030048327, 𝜆max = 12.007560699871851


### 4. Определить число обусловленности матрицы 

$\mu = ||A||\cdot||A^{-1}||$, $||A||=\sqrt{\sum_{i,j}(|a_{ij}|^2)}$ - норма Фробениуса

In [10]:
def сondition_number(matr):
    return norm(matr) * norm(np.linalg.inv(matr))

In [11]:
matrix_coeff = generate_matrix(n, a_i, b_i, c_i, p_i, f)[:n,:n]
mu = сondition_number(matrix_coeff)
print(f'𝜇 = ||A||⋅||inv(A)|| = {mu}')

𝜇 = ||A||⋅||inv(A)|| = 103.05143091950177


## Дополнительное задание
### Матрица коэффициентов не помещается в оперативную память

In [12]:
def Jacobi_method_limit_memory(m_diag, u_diag, l_diag, l_row, b_vect, norm_limit=1e-14):
    def sum_(i):
        if i == 0: return curr_x[i+1]*u_diag[i]
        if i == n-1:
            arr = l_row * curr_x
            arr[-1] = 0
            return np.sum(arr)
        return curr_x[i-1]*l_diag[i] + curr_x[i+1]*u_diag[i]
    
    n = len(m_diag)
    curr_x = np.ones(n)
    prev_x = np.copy(curr_x) + 1e10
    
    while norm(curr_x-prev_x) > norm_limit:
        prev_x = np.copy(curr_x)
        
        for i in range(n):
            curr_x[i] = 1/m_diag[i] * (b_vect[i]-sum_(i))
    
    return curr_x

In [13]:
# Выделение важнейших элементов матрицы коэффициентов - ненулевых диагоналей и строк
main_diag = np.diag(A_matr, 0) # массив из n элементов
upper_diag = np.diag(A_matr, 1) # массив из n-1 элементов
lower_diag = np.diag(A_matr, -1) # массив из n-1 элементов
lower_row = A_matr[-1,:] # массив из n элементов
# плюс массив свободных членов из n элементов

# итого данная система описывается ~5n элементами, вместо n^2 элементами

solution_Jacobi_lmt = Jacobi_method_limit_memory(main_diag, upper_diag, lower_diag, lower_row, b_vect)
print('Решение СЛАУ методом Якоби (с ограничением памяти):\n', solution_Jacobi_lmt)
print()

residual_Jacobi_lmt = np.dot(A_matr, solution_Jacobi_lmt)-b_vect
print('Вектор невязки для метода Якоби (с ограничением памяти):\n', residual_Jacobi_lmt)
print()

residual_Jacobi_norm_lmt = norm(residual_Jacobi_lmt)
print('Норма вектора невязки для метода Якоби (с ограничением памяти):\n', residual_Jacobi_norm_lmt)

Решение СЛАУ методом Якоби (с ограничением памяти):
 [ 8.33333333e-02  1.66666667e-01  2.50000000e-01  3.33333333e-01
  4.16666667e-01  5.00000000e-01  5.83333333e-01  6.66666667e-01
  7.50000000e-01  8.33333333e-01  9.16666667e-01  1.00000000e+00
  1.08333333e+00  1.16666667e+00  1.25000000e+00  1.33333333e+00
  1.41666667e+00  1.50000000e+00  1.58333333e+00  1.66666667e+00
  1.75000000e+00  1.83333333e+00  1.91666667e+00  2.00000000e+00
  2.08333333e+00  2.16666667e+00  2.25000000e+00  2.33333333e+00
  2.41666667e+00  2.50000000e+00  2.58333333e+00  2.66666667e+00
  2.75000000e+00  2.83333333e+00  2.91666667e+00  3.00000000e+00
  3.08333333e+00  3.16666667e+00  3.25000000e+00  3.33333333e+00
  3.41666667e+00  3.50000000e+00  3.58333333e+00  3.66666667e+00
  3.75000000e+00  3.83333333e+00  3.91666667e+00  4.00000000e+00
  4.08333333e+00  4.16666667e+00  4.25000000e+00  4.33333333e+00
  4.41666667e+00  4.50000000e+00  4.58333333e+00  4.66666667e+00
  4.75000000e+00  4.83333333e+00  4.9